## 6.3 LSTM 구현

- 한 단계만 처리하는 LSTM 클래스를 구현한 다음, 이 어서 T개의 단계를 한꺼번에 처리하면 LSTM을 구현한다.

![image](https://github.com/choibigo/Study/assets/38881179/7c78c190-7bfd-4f40-a9a6-438a6bd01e63)

![image](https://github.com/choibigo/Study/assets/38881179/7aa8b5ff-8f8a-42a1-b2b4-ad67bce69b45)

![image](https://github.com/choibigo/Study/assets/38881179/e6307081-20a9-411e-916d-240af846902e)

![image](https://github.com/choibigo/Study/assets/38881179/5af03eed-dc2c-4039-9c11-a96fbc5549a9)

- f : forget gate로 입력된 cell을 얼마나 '잊을'건지를 결정하는 연산이다, 이 결과가 입력 셀과 곱해져 입력 셀의 잊는 정도를 결정한다.
- g : Memory cell로 입력 데이터를 기억하기 위해 사용하는 연산이다.
- i : input gate로 입력 데이터에 대해 얼마나 기억할 지를 결정하는 연산이다, g 연산과 곱해져 입력데이터를 얼마나 기억할지 정도를 결정한다.
- o : output gate로 연산된 cell을 얼마나출력 할지 결정하는 연산이다, 이 cell과 곱해져 출력 셀을 얼마나 내보낼지 결정한다.
- ct(다음 출력할 cell) : forget gate와 이전 cell 의 원소별 곱 과 Memory cell과 Input Gate가 더해지는 연산이다.
- ht(hidden state): cell tanh가 곱해진 결과와 Output gate가 곱해지는 연산이다.


![image](https://github.com/choibigo/Study/assets/38881179/103277a1-c219-4b50-80d4-6684d3419784)

![image](https://github.com/choibigo/Study/assets/38881179/010b4d62-0dc8-4760-a8e6-c65dd619b1da)

- f, g, i, o 연산이 각각 수행되지만 모두 Affine 연산(곱이 더해진 형태)이다, 이를 행렬 곱으로 계산할 수 있다.
- 이를 통해 연산의 속도를 높일 수 있다.
- 각 연산이 필요한 Wx와 Wh를 하나의 행렬로 만들어 큰 Wx, Wh를 만든다.
- 연산이 끝난 후 Slice 연산을 통해 각 Affine 결과를 얻은 후, 각각의 활성화 함수를 거쳐서 최종적이 f, g, i, o를 얻을 수 있다.

In [1]:
import sys
sys.path.append('..')
from common.function import sigmoid

import numpy as np

class LSTM:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        # 초기 가중치 로서 앞서 말한 4개 연산에 대한 가중치가 담겨 있다

        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
    
    def forward(self, x, h_prev, c_prev):
        # 입력과 이전의 c, h를 입력으로 받는다.
        Wx, Wh, b = self.params
        N, H = h_prev.shape # 입력 데이터의 미니배치수 x hidden state 차원수
        A = np.matmul(x, Wx) + np.matmul(h_prev, Wh) + b # 행렬로 Affine 연산

        #slice
        f = A[:, :H]
        g = A[:, H:H*2]
        i = A[:, H*2:H*3]
        o = A[:, H*3:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i # 출력할 ct의 최종 연산
        h_next = o * np.tanh(c_next) # output gate와 c_next를 곱한 결과

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)

        return h_next, c_next
    
    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache
        
        tanh_c_next = np.tanh(c_next)
        
        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)
        
        dc_prev = ds * f
        
        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i
        
        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)
        
        dA = np.hstack((df, dg, di, do))
        
        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)
        
        return dx, dh_prev, dc_prev

![image](https://github.com/choibigo/Study/assets/38881179/d2878136-5b1e-4a18-a4f2-fe34feb7698d)

- N : 미니 배치 수
- D : 입력 데이터의 차원 수
- H : 은닉 상태의 차원 수
- f, g, i, o를 하나의 weight 행렬로 묶었기 때문에 LSTM 계층에서도 3개의 매개변수(Wx, Wh, b)만 관리하면 된다.
- 그러나 RNN과 각 Weight의 형상이 다르다.

#### 역전파

![image](https://github.com/choibigo/Study/assets/38881179/acb92725-1f07-4913-aa82-271cbbcce0a7)

- Slice는 나눠서 분배하는 연산이다, 반대로 역전파는 4개의 기울기를 결합 해야한다.
- DA의 연산은 각 기울기를 하나로 묶는 연산 이며 Numpy의 hstack() 메소드를 활용할 수 있다, 이 를 이용하여 배열을 가로로 연결할 수 있다.

#### 6.3.1 Time LSTM 구현
- TimeLSTM은 T개 분의 시계열 데이터를 한꺼번에 처리하는 계층이다.

![image](https://github.com/choibigo/Study/assets/38881179/3a1368e2-1a2b-4de8-b774-fbcf71975de9)

- Truncated BPTT는 역전파의 연결은 적당히 끊고 순전파의 흐름은 그대로 유지한다.
- 은닉 상태와 기억 셀을 인스턴스 변수로 유지하도록 한다.
- 이렇게 하면 forward()를 호출했을 때 이전 시각의 은닉 상태, 기억셀에서 부터 시작할 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/99031104-59e3-4d2b-a007-a553040537c6)

In [ ]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        self.h, self.c = None, None # Truncated BPTT를 사용하기 위해 이용하는 h와 c를 위한 Instance 변수
        self.stateful = stateful

    def forward(self, xs):
        # 적당한 크기의 시계열 데이터(T)가 주어진다.
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]

        self.layers = []
        hs = np.empty((N, T, H), dtype=np.float64) # 배치사이즈 x Truncated BPTT를 수행시 주어지는 데이터 개수 X hidden state 차원수

        if not self.stateful or self.h is None:
            # 1) state를 사용하지 않는 경우
            # 2) h가 없는 경우
            # h를 초기화 한다.
            self.h = np.zeros((N, H), dtype=np.float64)

        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype=np.float64)

        for t in range(T):
            layer = LSTM(*self.params) # LSTM을 T번 반복 한다.
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c) # t번째 데이터를 LSTM 1개에 통과 시킨후 hidden state와 cell을 얻는다.
            hs[:, t, :] = self.h # hs Update

            self.layers.append(layer) # backward 하기위해 Layer 정보 저장
        
        return hs # TimeLSTM의 최종 output은 전체 hs이다.
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype=np.float64)
        dh, dc = 0,0

        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t] # t번째 Layer
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
            
            for i, grad in enumerate(grads):
                self.grads[i][...] = grad

        self.dh = dh
        return dxs # 최종 backpropa
    
    def set_state(self, h, c=None):
        self.h = h
        self.c = c

    def reset_state(self):
        self.h = None
        self.c = None